<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Mnist_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook

     |████████████████████████████████| 727kB 6.7MB/s 
     |████████████████████████████████| 194kB 8.8MB/s 
     |████████████████████████████████| 1.2MB 12.3MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/0'),Path('/root/.fastai/data/mnist_png/training/8'),Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/4'),Path('/root/.fastai/data/mnist_png/training/2')]

In [5]:
folder = 'training/3'

(path/folder).ls()

(#6131) [Path('/root/.fastai/data/mnist_png/training/3/25245.png'),Path('/root/.fastai/data/mnist_png/training/3/58707.png'),Path('/root/.fastai/data/mnist_png/training/3/27140.png'),Path('/root/.fastai/data/mnist_png/training/3/10214.png'),Path('/root/.fastai/data/mnist_png/training/3/33594.png'),Path('/root/.fastai/data/mnist_png/training/3/45836.png'),Path('/root/.fastai/data/mnist_png/training/3/6920.png'),Path('/root/.fastai/data/mnist_png/training/3/51091.png'),Path('/root/.fastai/data/mnist_png/training/3/17634.png'),Path('/root/.fastai/data/mnist_png/training/3/7295.png')...]

In [6]:
(path/'testing').ls()

(#10) [Path('/root/.fastai/data/mnist_png/testing/1'),Path('/root/.fastai/data/mnist_png/testing/9'),Path('/root/.fastai/data/mnist_png/testing/0'),Path('/root/.fastai/data/mnist_png/testing/8'),Path('/root/.fastai/data/mnist_png/testing/7'),Path('/root/.fastai/data/mnist_png/testing/5'),Path('/root/.fastai/data/mnist_png/testing/3'),Path('/root/.fastai/data/mnist_png/testing/6'),Path('/root/.fastai/data/mnist_png/testing/4'),Path('/root/.fastai/data/mnist_png/testing/2')]

In [7]:
train_x = []
train_y = []
test_x = []
test_y = []
for digit in range(10):
    folder = 'training/' + str(digit)
    new_inp = [tensor(Image.open(x)) for x in (path/folder).ls()]
    train_x += new_inp
    train_y += [digit]*len(new_inp)
    test_folder = 'testing/' + str(digit)
    test_inp = [tensor(Image.open(x)) for x in (path/test_folder).ls()]
    test_x += test_inp
    test_y += [digit]*len(test_inp)
    

In [8]:
train_x_stacked = torch.stack(train_x).view(-1, 28*28).float()/255
train_y_stacked = tensor(train_y).view(-1, 1)
train_ds = list(zip(train_x_stacked, train_y_stacked))
train_dl = DataLoader(train_ds, batch_size= 256, shuffle=True)

test_x_stacked = torch.stack(test_x).view(-1, 28*28).float()/255
test_y_stacked = tensor(test_y).view(-1, 1)
test_ds = list(zip(test_x_stacked, test_y_stacked))
test_dl = DataLoader(test_ds, batch_size=256)

dls = DataLoaders(train_dl, test_dl)

train_x_stacked.shape, train_y_stacked.shape, test_x_stacked.shape, test_y_stacked.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [9]:
train_x[2000]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  13, 101, 101, 208, 255, 119,  95,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  68, 207, 253, 253, 253, 253, 253, 250, 130,   7,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 174, 253, 253, 253, 253, 253, 253, 2

In [10]:
df = pd.DataFrame(torch.trunc(tensor(train_x[2000]).float()))
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.000000,101.000000,101.000000,208.000000,255.000000,119.000000,95.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,68.000000,207.000000,253.000000,253.000000,253.000000,253.000000,253.000000,250.000000,130.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,174.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,93.000000,244.000000,253.000000,253.000000,122.000000,115.000000,160.000000,186.000000,253.000000,253.000000,187.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.000000,253.000000,253.000000,253.000000,153.000000,6.000000,0.000000,0.000000,17.000000,70.000000,232.000000,201.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,124.000000,253.000000,253.000000,253.000000,123.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53.000000,233.000000,175.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
def batch_accuracy(x, y):
    preds = torch.argmax(x, 1)
    correct = preds == y
    return correct.float().mean()

simple_net = nn.Sequential(
            nn.Linear(28*28, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 10)
)
loss = CrossEntropyLossFlat()
learn = Learner(dls, simple_net, opt_func = SGD, loss_func = loss, metrics = batch_accuracy)
learn.fit(40, 0.3)


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.358166,0.303789,0.834277,00:01
1,0.222474,0.190920,0.865857,00:01
2,0.168673,0.169982,0.871329,00:01
3,0.139638,0.137890,0.880031,00:01
4,0.118034,0.130798,0.880909,00:01
5,0.107984,0.159387,0.875085,00:01
6,0.095751,0.160191,0.873549,00:01
7,0.091912,0.107333,0.889243,00:01
8,0.081185,0.127012,0.884239,00:01
9,0.071906,0.110794,0.886895,00:01


In [ ]:
x,y = dls.one_batch()
learn.get_preds(dl = [(x,y)])

In [13]:
??Learner

In [90]:
def init_params(size1, size2): return torch.randn(size1, size2).requires_grad_()

def my_linear(weights, bias, x): return x@weights + bias

def my_relu(x): return torch.maximum(tensor(0), x)

def my_softmax(x): 
    # Normalize the input to avoid numerical problems
    x = x - torch.max(x, dim = 1).values.view(-1, 1)
    return torch.exp(x) / torch.exp(x).sum(dim = 1).unsqueeze(1)

def my_centloss(x, y): 
    x = my_softmax(x)
    y = y.view(-1, 1)
    selection = torch.gather(x, 1, y)
    #print(selection)
    return (1-selection).mean()



In [95]:
w1 = init_params(28*28, 20)
b1 = init_params(1, 1)
w2 = init_params(20, 10)
b2 = init_params(1, 1)
lr = 1e-1
params = w1, b1, w2, b2

In [96]:
type(params)

tuple

In [104]:
def my_model(x):
    lay1 = my_linear(w1, b1, x)
    nonlin = my_relu(lay1)
    preds = my_linear(w2, b2, nonlin)
    return(preds)

def update_grads(preds, y, params):
    loss = my_centloss(preds, y)
    loss.backward()
    for p in params:
        p.data -= p.grad*lr
        p.grad = None

def eval_batch():
    preds = my_model(test_x_stacked)
    positions = torch.max(preds, dim = 1).indices
    return (positions == test_y_stacked).float().mean()


In [106]:
eval_batch()

tensor(0.0984)

In [55]:
train_x_stacked[:2].shape, train_y_stacked[:2].shape

(torch.Size([2, 784]), torch.Size([2, 1]))

In [94]:
lvl1 = my_linear(w1, b1, train_x_stacked[:2])
nonlin = my_relu(lvl1)
pred = my_linear(w2, b2, nonlin)
loss = my_centloss(pred, train_y_stacked[:2])
loss.backward()
w1.data -= w1.grad*lr
b1.data -= b1.grad*lr
w2.data -= w2.grad*lr
b2.data -= b2.grad*lr
w1.grad = None
b1.grad = None
w2.grad = None
b2.grad = None

In [84]:
w1.grad.data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  """Entry point for launching an IPython kernel.


AttributeError: ignored

In [74]:
pred.shape
torch.exp(pred)
torch.max(pred, dim = 1).values.view(-1, 1)

tensor([[126.2350],
        [ 91.0678]], grad_fn=<ViewBackward>)

In [75]:
pred - torch.max(pred, dim = 1).values.view(-1, 1)

tensor([[   0.0000,  -73.9121, -126.6762,  -65.5008, -125.7796, -147.9242, -130.4742, -108.4133, -147.9715, -147.6310],
        [   0.0000,  -74.6761,  -84.4303,  -55.9350,  -90.3402, -100.4288, -111.5041,  -77.1204, -107.2406,  -87.5272]], grad_fn=<SubBackward0>)

In [59]:
pred

tensor([[126.2350,  52.3229,  -0.4412,  60.7342,   0.4554, -21.6892,  -4.2392,  17.8218, -21.7365, -21.3959],
        [ 91.0678,  16.3917,   6.6375,  35.1328,   0.7276,  -9.3610, -20.4363,  13.9474, -16.1729,   3.5406]], grad_fn=<AddBackward0>)

In [15]:
test_tensor = tensor([1,-2,3], [-2,7,4]).float()
test_tensor.shape

torch.Size([2, 3])

In [16]:
(torch.exp(test_tensor) / torch.exp(test_tensor).sum(dim = 1).unsqueeze(1)).sum(1)

tensor([1., 1.])

In [46]:
my_softmax(tensor([1,1,3], [-2,7,1]).float())
(tensor(1) - tensor([[0.1065], [0.0025]])).mean()

tensor(0.9455)

In [43]:
my_centloss(tensor([1,1,3], [-2,7,1]).float(), tensor([1,2]))

tensor([[0.1065],
        [0.0025]])


tensor(0.9455)

In [26]:
test_tensor = torch.tensor([[1,-2,3], [-2,7,4], [1,1,1]]).float()
select_tensor = torch.tensor([1,2, 0], dtype=torch.int64).view(-1,1) # number of dimension should match with the test tensor.
final_tensor = torch.gather(test_tensor, 1, select_tensor)
final_tensor

tensor([[-2.],
        [ 4.],
        [ 1.]])

In [27]:
torch.gather(test_tensor, 0, select_tensor)

tensor([[-2.],
        [ 1.],
        [ 1.]])